In [ ]:
import sys

sys.path.append("..")
from piece import piece

piece.start(should_send_to_score=False)

In [ ]:
from enum import StrEnum, Enum
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
import random
from soundmining_tools.generative import *
import math
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.modular.synth_player import SynthNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools.modular import instrument
from soundmining_tools import supercollider_client

SOUND_PATH = (
    "/Users/danielstahl/Documents/Music/Pieces/Concrete Music/Concrete Music 13/sounds/Concrete Music 13_sounds"
)
IR_PATH = "/Users/danielstahl/Documents/Music/impulse-response/convolution-ir"

Sound = StrEnum("Sound", ["WOOD_DAMPED_KNIFE_HIT", "WOOD_KNIFE_HIT", "WOOD_KNUCKLE_HIT"])


colors = {
    Sound.WOOD_DAMPED_KNIFE_HIT: "blue",
    Sound.WOOD_KNIFE_HIT: "red",
    Sound.WOOD_KNUCKLE_HIT: "black"
}

piece.reset()
(
    piece.synth_player
        .add_sound(Sound.WOOD_DAMPED_KNIFE_HIT, f"{SOUND_PATH}/Wood damped knife hit.aif", 0.308, 0.591)
        .add_sound(Sound.WOOD_KNIFE_HIT, f"{SOUND_PATH}/Wood knife hit.aif", 0.336, 0.644)
        .add_sound(Sound.WOOD_KNUCKLE_HIT, f"{SOUND_PATH}/Wood knuckle hit.aif", 0.412, 0.679)        
        .start()
)
if piece.synth_player.should_send_to_score:
    score = piece.synth_player.supercollider_score
    score.add_message(supercollider_client.group_head(0, instrument.NodeId.SOURCE.value))
    score.add_message(supercollider_client.group_tail(NodeId.SOURCE.value, NodeId.EFFECT.value))
    score.add_message(supercollider_client.group_tail(NodeId.EFFECT.value, NodeId.ROOM_EFFECT.value))
    score.add_message(supercollider_client.load_dir(instrument.DEFAULT_SYNTH_DIR))
    score.add_message(supercollider_client.alloc_read(0, f"{SOUND_PATH}/Wood damped knife hit.aif"))
    score.add_message(supercollider_client.alloc_read(1, f"{SOUND_PATH}/Wood knuckle hit.aif"))
    score.add_message(supercollider_client.alloc_read(2, f"{SOUND_PATH}/terrys_typing_ortf-L.wav"))

SoundType = StrEnum("SoundType", ["LOW", "MIDDLE", "HIGH"])

sounds = {
    Sound.WOOD_DAMPED_KNIFE_HIT: {
        SoundType.LOW: [117, 258],
        SoundType.MIDDLE: [725, 1124],
        SoundType.HIGH: [1382, 1454, 1709, 1873, 2110, 2274, 2625, 3000, 3165, 3467, 4358, 4802, 5392, 6234, 7009, 7758, 8320, 8601]
    },
    Sound.WOOD_KNIFE_HIT: {
        SoundType.LOW: [117, 259, 279],
        SoundType.MIDDLE: [445, 727, 888, 1031, 1197],
        SoundType.HIGH: [1287, 1382, 1431, 1523, 1688, 1804, 2041, 2204, 2435, 2695, 2951, 3305, 3727, 4266, 4969, 5812, 6351, 7291, 8225]
    },
    Sound.WOOD_KNUCKLE_HIT: {
        SoundType.LOW: [ 116, 256, 398],
        SoundType.MIDDLE: [750, 1127],
        SoundType.HIGH: [1266, 1337, 1501, 1734, 1852, 1946, 2228, 2414, 2555, 2859, 3422, 3773, 4221, 4852, 5484, 6094, 6772, 7499, 7828, 8366]
    }
}

pan_points = {
    SoundType.LOW: [(-0.25, 0), (0, 0.25)],
    SoundType.MIDDLE: [(-0.66, -0.33), (0.33, 0.66)],
    SoundType.HIGH: [(-0.99, -0.75), (0.75, 0.99)]
}

pan_dinstance = {
    SoundType.LOW: 0.25,
    SoundType.MIDDLE: 0.5,
    SoundType.HIGH: 1.0
}

LOW_SOUND_EFFECT = 0
LOW_SOUND_CLEAN = 2
LOW_DUST_EFFECT = 4
LOW_DUST_CLEAN = 6
LOW_NOISE_EFFECT = 8
LOW_NOISE_CLEAN = 10

HIGH_SOUND_EFFECT = 12
HIGH_SOUND_CLEAN = 14
HIGH_DUST_EFFECT = 16
HIGH_DUST_CLEAN = 18
HIGH_NOISE_EFFECT = 20
HIGH_NOISE_CLEAN = 22

class LowSoundClouds:
    
    def make_pan_line_chain():
        return MarkovChain(
            {
                True: {True: 0.5, False: 0.5},
                False: {True: 0.5, False: 0.5}
            }, True)
    
    def make_sound_type_chain():
        return MarkovChain({
            SoundType.HIGH: {SoundType.HIGH: 0.0, SoundType.MIDDLE: 0.5, SoundType.LOW: 0.5},
            SoundType.MIDDLE: {SoundType.HIGH: 0.2, SoundType.MIDDLE: 0.3, SoundType.LOW: 0.5},
            SoundType.LOW: {SoundType.HIGH: 0.2, SoundType.MIDDLE: 0.5, SoundType.LOW: 0.3},
        }, SoundType.HIGH)
    
    sound_type_chain = make_sound_type_chain()
    dust_type_chain = make_sound_type_chain()
    dust_pan_line_chain = make_pan_line_chain()
    noise_type_chain = make_sound_type_chain()
    noise_pan_line_chain = make_pan_line_chain()

    @classmethod
    def make_sound_cloud_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_free_reverb(piece.control_instruments.static_control(1.0), mix=1.0, room=0.3, damp=0.3)
                .play(start_time, duration, output_bus=LOW_SOUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=LOW_SOUND_CLEAN)
        )
        return effect

    @classmethod
    def make_dust_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_free_reverb(piece.control_instruments.static_control(1.0), mix=1.0, room=0.5, damp=0.6)
                .play(start_time, duration, output_bus=LOW_DUST_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=LOW_DUST_CLEAN)
        )
        return effect
    
    @classmethod
    def make_noise_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_g_verb(piece.control_instruments.static_control(1.0), roomsize=30, revtime=5, damping=0.6)
                .play(start_time, duration, output_bus=LOW_NOISE_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=LOW_NOISE_CLEAN)
        )
        return effect
    
    @classmethod
    def play_sound_cloud(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        current_time = start_time
        cloud_duration = random_range(13, 21)
        end_time = start_time + cloud_duration
        effect = cls.make_sound_cloud_effect(start_time, cloud_duration + 1.0)

        while current_time < end_time:
            sound_type = cls.sound_type_chain.next()
            pan = pan_point(pan_points[sound_type])
            pitch = random.choice(sounds[sound][sound_type])
            bw = random_range(100, 200)
            static_amp_factor = 1 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
            (
                piece.synth_player.note()
                .sound_mono(sound, 1.0, piece.control_instruments.static_control(1.0))
                .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                .mono_volume(piece.control_instruments.static_control(amp_factor))
                .pan(piece.control_instruments.static_control(pan))                    
                #.play(start_time=current_time)
                .send_to_synth_note(effect, start_time=current_time)
            )
            notes.append(SequenceNote(start=current_time, track="Low Sound", duration=0.1, freq=pitch, color=colors[sound]))
            current_time += random_range(0.7, 1.9)
        return notes


    @classmethod
    def play_dust_cloud(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        effect = cls.make_dust_effect(start_time, 21)
        for _ in range(random_int_range(5, 7)):
            sound_type = cls.dust_type_chain.next()
            pitch = random.choice(sounds[sound][sound_type])
            if cls.dust_pan_line_chain.next():
                pan_start, pan_end = pan_line(pan_dinstance[sound_type], pan_points[sound_type])
                pan_control = piece.control_instruments.line_control(pan_start, pan_end)
            else:
                pan_pos = pan_point(pan_points[sound_type])
                pan_control = piece.control_instruments.static_control(pan_pos)                
            
            bw = random_range(100, 200)
            static_amp_factor = 10 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
            duration = random_range(13, 21)
            (
                piece.synth_player.note()
                .dust(piece.control_instruments.static_control(0.2), piece.control_instruments.sine_control(0, 1))
                .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                .mono_volume(piece.control_instruments.static_control(amp_factor))            
                .pan(pan_control)
                #.play(start_time=start_time, duration=duration)
                .send_to_synth_note(effect, start_time, duration)
            )
            notes.append(SequenceNote(start=start_time, track="Low Dust", duration=duration, freq=pitch, color=colors[sound]))
        return notes

    @classmethod
    def play_noise_chord(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        current_time = start_time
        effect = cls.make_noise_effect(start_time, 21 + (2 * 7))
        for _ in range(random_int_range(5, 7)):                
            sound_type = cls.noise_type_chain.next()
            pitch = random.choice(sounds[sound][sound_type])
            bw = random_range(100, 300)
            static_amp_factor = 0.5 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor                                
            if cls.noise_pan_line_chain.next():
                pan_start, pan_end = pan_line(pan_dinstance[sound_type], pan_points[sound_type])
                pan_control = piece.control_instruments.line_control(pan_start, pan_end)
            else:
                pan_pos = pan_point(pan_points[sound_type])
                pan_control = piece.control_instruments.static_control(pan_pos)
            duration = random_range(13, 21)    
            (        
                piece.synth_player.note()                        
                    .white_noise(piece.control_instruments.sine_control(0, 0.5))
                    .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                    .mono_volume(piece.control_instruments.static_control(amp_factor))                        
                    .mono_low_pass_filter(piece.control_instruments.static_control(pitch))
                    .mono_high_pass_filter(piece.control_instruments.static_control(pitch))                                                    
                    .pan(pan_control)
                    #.play(start_time=current_time, duration=duration)
                    .send_to_synth_note(effect, start_time, duration)
            )
            notes.append(SequenceNote(start=current_time, track="Low Noise", duration=duration, freq=pitch, color=colors[sound]))
            current_time += random_range(1, 2)
        return notes

class HighSoundClouds:

    def make_pan_line_chain():
        return MarkovChain(
            {
                True: {True: 0.5, False: 0.5},
                False: {True: 0.5, False: 0.5}
            }, True)
    
    def make_sound_type_chain():
        return MarkovChain({
            SoundType.HIGH: {SoundType.HIGH: 0.8, SoundType.MIDDLE: 0.1, SoundType.LOW: 0.1},
            SoundType.MIDDLE: {SoundType.HIGH: 0.9, SoundType.MIDDLE: 0.0, SoundType.LOW: 0.1},
            SoundType.LOW: {SoundType.HIGH: 0.9, SoundType.MIDDLE: 0.1, SoundType.LOW: 0.0},
        }, SoundType.HIGH)
    
    sound_type_chain = make_sound_type_chain()
    dust_type_chain = make_sound_type_chain()
    dust_pan_line_chain = make_pan_line_chain()
    noise_type_chain = make_sound_type_chain()
    noise_pan_line_chain = make_pan_line_chain()

    @classmethod
    def make_sound_cloud_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_free_reverb(piece.control_instruments.static_control(1.0), mix=1.0, room=0.2, damp=0.3)
                .play(start_time, duration, output_bus=HIGH_SOUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=HIGH_SOUND_CLEAN)
        )
        return effect
    
    @classmethod
    def make_dust_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_free_reverb(piece.control_instruments.static_control(1.0), mix=1.0, room=0.2, damp=0.6)
                .play(start_time, duration, output_bus=HIGH_DUST_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=HIGH_DUST_CLEAN)
        )
        return effect
    
    @classmethod
    def make_noise_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_g_verb(piece.control_instruments.static_control(1.0), roomsize=30, revtime=5, damping=0.4)
                .play(start_time, duration, output_bus=HIGH_NOISE_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=HIGH_NOISE_CLEAN)
        )
        return effect
    
    @classmethod
    def play_sound_cloud(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        current_time = start_time
        cloud_duration = random_range(13, 21)
        end_time = start_time + cloud_duration
        effect = cls.make_sound_cloud_effect(start_time, cloud_duration + 1.0)
        while current_time < end_time:
            sound_type = cls.sound_type_chain.next()
            pan = pan_point(pan_points[sound_type])
            pitch = random.choice(sounds[sound][sound_type])
            bw = random_range(100, 200)
            static_amp_factor = 1 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
            (
                piece.synth_player.note()
                .sound_mono(sound, 1.0, piece.control_instruments.static_control(1.0))
                .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                .mono_volume(piece.control_instruments.static_control(amp_factor))
                .pan(piece.control_instruments.static_control(pan))                    
                #.play(start_time=current_time)
                .send_to_synth_note(effect, start_time=current_time)
            )
            notes.append(SequenceNote(start=current_time, track="High Sound", duration=0.1, freq=pitch, color=colors[sound]))
            current_time += random_range(0.01, 0.9)
        return notes
    
    @classmethod
    def play_dust_cloud(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        effect = cls.make_dust_effect(start_time, 21)
        for _ in range(random_int_range(5, 7)):
            sound_type = cls.dust_type_chain.next()
            pitch = random.choice(sounds[sound][sound_type])
            if cls.dust_pan_line_chain.next():
                pan_start, pan_end = pan_line(pan_dinstance[sound_type], pan_points[sound_type])
                pan_control = piece.control_instruments.line_control(pan_start, pan_end)
            else:
                pan_pos = pan_point(pan_points[sound_type])
                pan_control = piece.control_instruments.static_control(pan_pos)                
            
            bw = random_range(100, 200)
            static_amp_factor = 10 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
            duration = random_range(13, 21)
            (
                piece.synth_player.note()
                .dust(piece.control_instruments.static_control(1), piece.control_instruments.sine_control(0, 1))
                .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                .mono_volume(piece.control_instruments.static_control(amp_factor))            
                .pan(pan_control)
                #.play(start_time=start_time, duration=duration)
                .send_to_synth_note(effect, start_time, duration)
            )
            notes.append(SequenceNote(start=start_time, track="High Dust", duration=duration, freq=pitch, color=colors[sound]))
        return notes
    
    @classmethod
    def play_noise_chord(cls, sound: Sound, start_time: float) -> list[SequenceNote]:
        notes = []
        current_time = start_time
        effect = cls.make_noise_effect(start_time, 21 + (7 * 2))
        for _ in range(random_int_range(5, 7)):                
            sound_type = cls.noise_type_chain.next()
            pitch = random.choice(sounds[sound][sound_type])
            bw = random_range(100, 300)
            static_amp_factor = 0.5 * random_range(0.85, 1.15)
            rq = bw / pitch
            amp_factor = (1 / math.sqrt(rq)) * static_amp_factor                                
            if cls.noise_pan_line_chain.next():
                pan_start, pan_end = pan_line(pan_dinstance[sound_type], pan_points[sound_type])
                pan_control = piece.control_instruments.line_control(pan_start, pan_end)
            else:
                pan_pos = pan_point(pan_points[sound_type])
                pan_control = piece.control_instruments.static_control(pan_pos)
            duration = random_range(13, 21)    
            (        
                piece.synth_player.note()                        
                    .white_noise(piece.control_instruments.sine_control(0, 0.5))
                    .mono_band_pass_filter(piece.control_instruments.static_control(pitch), piece.control_instruments.static_control(rq))
                    .mono_volume(piece.control_instruments.static_control(amp_factor))                        
                    .mono_low_pass_filter(piece.control_instruments.static_control(pitch))
                    .mono_high_pass_filter(piece.control_instruments.static_control(pitch))                                                    
                    .pan(pan_control)
                    #.play(start_time=current_time, duration=duration)
                    .send_to_synth_note(effect, current_time, duration)
            )
            notes.append(SequenceNote(start=start_time, track="High Noise", duration=duration, freq=pitch, color=colors[sound]))
            current_time += random_range(1, 2)
        return notes

class ShortSoundHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)
        
    
    def handle_note(self, patch_arguments: PatchArguments) -> None:          
        match patch_arguments.midi_note:
            case 48:
                HighSoundClouds.play_sound_cloud(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)
            case 49:                 
                HighSoundClouds.play_sound_cloud(Sound.WOOD_KNIFE_HIT, patch_arguments.start)            
            case 50:                
                HighSoundClouds.play_sound_cloud(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)
            case 51:
                HighSoundClouds.play_dust_cloud(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)                        
            case 52:                 
                HighSoundClouds.play_dust_cloud(Sound.WOOD_KNIFE_HIT, patch_arguments.start)
            case 53:                
                HighSoundClouds.play_dust_cloud(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)
            case 54:                
                HighSoundClouds.play_noise_chord(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)
            case 55:                 
                HighSoundClouds.play_noise_chord(Sound.WOOD_KNIFE_HIT, patch_arguments.start)
            case 56:                
                HighSoundClouds.play_noise_chord(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)

            case 60:                
                LowSoundClouds.play_sound_cloud(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)
            case 61:                 
                LowSoundClouds.play_sound_cloud(Sound.WOOD_KNIFE_HIT, patch_arguments.start)
            case 62:                
                LowSoundClouds.play_sound_cloud(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)
            case 63:
                LowSoundClouds.play_dust_cloud(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)                        
            case 64:                 
                LowSoundClouds.play_dust_cloud(Sound.WOOD_KNIFE_HIT, patch_arguments.start)
            case 65:                
                LowSoundClouds.play_dust_cloud(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)
            case 66:                
                LowSoundClouds.play_noise_chord(Sound.WOOD_DAMPED_KNIFE_HIT, patch_arguments.start)
            case 67:                 
                LowSoundClouds.play_noise_chord(Sound.WOOD_KNIFE_HIT, patch_arguments.start)
            case 68:                
                LowSoundClouds.play_noise_chord(Sound.WOOD_KNUCKLE_HIT, patch_arguments.start)
            
my_handler = ShortSoundHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)

CloudType = StrEnum("CloudType", ["LOW", "HIGH"])

parts = [
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),

    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),

    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),


    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),



    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),

    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(1, 2)),

    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(1, 2)),
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(1, 2)),


    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_KNIFE_HIT, random_range(8, 13)),
    (CloudType.LOW, Sound.WOOD_KNUCKLE_HIT, random_range(8, 13)),
    (CloudType.HIGH, Sound.WOOD_DAMPED_KNIFE_HIT, random_range(8, 13)),
]

def step_handler(i: int, start: float) -> list[SequenceNote]:
    notes = []
    cloud_type, sound, _ = parts[i]
    match cloud_type:
        case CloudType.HIGH:
            notes.extend(HighSoundClouds.play_sound_cloud(sound, start))
            notes.extend(HighSoundClouds.play_dust_cloud(sound, start))
            notes.extend(HighSoundClouds.play_noise_chord(sound, start)) 
        case CloudType.LOW:
            notes.extend(LowSoundClouds.play_sound_cloud(sound, start))
            notes.extend(LowSoundClouds.play_dust_cloud(sound, start))
            notes.extend(LowSoundClouds.play_noise_chord(sound, start))
    return notes 

sequenser = (
    Sequencer(len(parts))
        .add_step_handler(step_handler)
        .next_time_handler(lambda i: parts[i][2]))

notes = sequenser.generate(0)
#notes = []

if piece.synth_player.should_send_to_score:
    piece.synth_player.supercollider_score.make_score_file("concrete-music-13-v1.txt")

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

min_freq = 0
max_freq = 0

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
        min_freq = min(min_freq, note.freq)
        max_freq = max(max_freq, note.freq)

    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 3
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()

@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)


with hold_canvas():

    canvas.clear()
    for track_index, track in enumerate(sorted(ui_piece.tracks, key=lambda tr: tr.track_name)):
        canvas.font = "14px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")

def stop_playback(b):
    piece.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)


In [ ]:
piece.stop()